In [218]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import os

In [219]:
path = "../output/kmeans/"
df_each = []
for filename in os.listdir(path):
    f = os.path.join(path, filename)
    if os.path.isfile(f):
        temp = pd.read_csv(f)
        temp = temp[temp.name.apply(lambda x: x != "name")]
        df_each.append((temp, filename))

In [220]:
print(len(df_each))
first_df = df_each[0][0]
first_df.columns

36


Index(['name', 'run_num', 'params', 'reduction_time', 'train_time', 'accuracy',
       'original_std_sum', 'original_std_mean', 'original_std_median',
       'original_std_max', 'original_std_min', 'original_sparsity',
       'original_shape', 'transformed_std_sum', 'transformed_std_mean',
       'transformed_std_median', 'transformed_std_max', 'transformed_std_min',
       'transformed_sparsity', 'transformed_shape', 'characteristics'],
      dtype='object')

In [238]:
def score(reduc_time, accuracy, train_time, red_w = 200, ac_w = 1, train_w = 20):
    return (accuracy ** ac_w)*(red_w/int(reduc_time + 1) + train_w/int(train_time + 1))

In [222]:
first_df.describe()

,run_num,reduction_time,train_time,accuracy,original_std_sum,original_std_mean,original_std_median,original_std_max,original_std_min,original_sparsity,transformed_std_sum,transformed_std_mean,transformed_std_median,transformed_std_max,transformed_std_min,transformed_sparsity
count,286.0,286.000000,286.000000,286.000000,2.860000e+02,286.000000,286.000000,286.000000,286.000000,286.000000,2.860000e+02,286.000000,286.000000,286.000000,286.000000,286.000000
mean,0.0,5.891288,1.066414,0.365874,9.903799e+06,819.279004,819.148145,862.744322,778.062290,0.330537,4.999442e+07,3961.474781,3968.625687,4894.990559,217.118101,0.114281
std,0.0,12.038355,1.571849,0.298715,6.523014e+06,1.141521,1.153206,33.370571,29.393450,0.001804,6.270752e+07,3421.741117,3426.567712,4537.349016,176.287417,0.157395
min,0.0,0.000011,0.169322,0.060000,8.191183e+05,817.067737,816.867522,836.085860,720.537873,0.326400,5.877160e+05,460.810237,461.065846,526.451980,25.498636,0.000000
25%,0.0,0.125975,0.258444,0.140000,8.210849e+05,818.670627,818.605567,839.702874,740.220146,0.329850,8.271562e+06,1558.938337,1559.702542,1843.760449,88.447957,0.000000
50%,0.0,1.471981,0.385439,0.220000,8.196896e+06,819.451670,819.440135,842.956882,792.928809,0.330620,2.617965e+07,2569.391709,2585.804514,2830.171832,142.053597,0.000000
75%,0.0,4.692783,1.380918,0.570000,1.638903e+07,819.870435,819.881129,910.982119,799.379351,0.332035,5.403078e+07,5531.386398,5533.124690,6751.795911,311.322959,0.327461
max,0.0,72.678207,13.931473,1.000000,1.641059e+07,821.084850,820.545799,920.261479,800.884973,0.333200,2.799809e+08,13922.473090,13995.318172,18937.284759,774.227713,0.352439


In [223]:
grouped_df = first_df.groupby(by=["name", "original_shape"])
grouped_df.apply(lambda a: a[:])[["name", "params", "reduction_time", "accuracy"]]

name  \
name                original_shape                            
JL transform        (1000, 1000)   16          JL transform   
                                   17          JL transform   
                                   18          JL transform   
                                   19          JL transform   
                                   20          JL transform   
...                                                     ...   
sparse JL transform (20000, 20000) 271  sparse JL transform   
                                   272  sparse JL transform   
                                   273  sparse JL transform   
                                   274  sparse JL transform   
                                   275  sparse JL transform   

                                                          params  \
name                original_shape                                 
JL transform        (1000, 1000)   16   {'ep': 0.05, 'de': 0.05}   
                                   17    {'ep': 0.05, 'de': 0.1}   
                                   18    {'ep': 0.1, 'de': 0.05}   
                                   19     {'ep': 0.1, 'de': 0.1}   
                                   20    {'ep': 0.5, 'de': 0.05}   
...                                                          ...   
sparse JL transform (20000, 20000) 271    {'ep': 0.1, 'de': 0.1}   
                                   272   {'ep': 0.5, 'de': 0.05}   
                                   273    {'ep': 0.5, 'de': 0.1}   
                                   274   {'ep': 0.9, 'de': 0.05}   
                                   275    {'ep': 0.9, 'de': 0.1}   

                                        reduction_time  accuracy  
name                original_shape                                
JL transform        (1000, 1000)   16         0.022519      0.18  
                                   17         0.014319      0.17  
                                   18         0.008627      0.14  
                                   19         0.010589      0.21  
                                   20         0.037107      0.26  
...                                                ...       ...  
sparse JL transform (20000, 20000) 271       44.043004      0.57  
                                   272        3.912686      0.12  
                                   273        4.327864      0.11  
                                   274        3.212227      0.13  
                                   275        2.631185      0.08  

[286 rows x 4 columns]

In [224]:
min_ind = grouped_df["reduction_time"].idxmin()
print(min_ind)
min_ind.to_numpy()

name                           original_shape
JL transform                   (1000, 1000)       18
                               (1000, 10000)      48
                               (1000, 20000)      75
                               (10000, 1000)     101
                               (10000, 10000)    127
                               (10000, 20000)    153
                               (20000, 1000)     179
                               (20000, 10000)    205
                               (20000, 20000)    231
Nothing                        (1000, 1000)       25
                               (1000, 10000)      51
                               (1000, 20000)      77
                               (10000, 1000)     103
                               (10000, 10000)    129
                               (10000, 20000)    155
                               (20000, 1000)     181
                               (20000, 10000)    207
                               (20000, 20000)    285


array([ 18,  48,  75, 101, 127, 153, 179, 205, 231,  25,  51,  77, 103,
       129, 155, 181, 207, 285,  24,  50,  76, 102, 128, 154, 180, 206,
       232,   4,  30,  56,  84, 111, 136, 163, 188, 240,  14,  40,  67,
        93, 119, 145, 164, 197, 249], dtype=int64)

In [225]:
first_df.iloc[min_ind]

,name,run_num,params,reduction_time,train_time,accuracy,original_std_sum,original_std_mean,original_std_median,original_std_max,...,original_sparsity,original_shape,transformed_std_sum,transformed_std_mean,transformed_std_median,transformed_std_max,transformed_std_min,transformed_sparsity,transformed_shape,characteristics
18,JL transform,0,"{'ep': 0.1, 'de': 0.05}",0.008627,0.217366,0.14,8.210849e+05,821.084850,820.420886,918.643900,...,0.326400,"(1000, 1000)",2.844116e+06,2562.266288,2580.266725,3179.313344,142.039525,0.000000,"(1110, 100)","{'n': 1000, 'd': 1000, 'a': -100, 'b': 100, 'c..."
48,JL transform,0,"{'ep': 0.9, 'de': 0.05}",0.091399,0.213023,0.14,8.198704e+05,819.870435,819.881129,842.956882,...,0.329850,"(1000, 10000)",9.529599e+06,8585.224536,8661.964974,10894.259866,457.587007,0.000000,"(1110, 88)","{'n': 1000, 'd': 10000, 'a': -100, 'b': 100, '..."
75,JL transform,0,"{'ep': 0.9, 'de': 0.1}",0.156952,0.245644,0.20,8.191183e+05,819.118324,819.011181,836.085860,...,0.330980,"(1000, 20000)",1.533993e+07,13819.759049,13949.287571,18286.243271,725.986361,0.000000,"(1110, 68)","{'n': 1000, 'd': 20000, 'a': -100, 'b': 100, '..."
101,JL transform,0,"{'ep': 0.9, 'de': 0.1}",0.043136,0.256056,0.11,8.174593e+06,817.459278,817.111549,920.261479,...,0.332900,"(10000, 1000)",3.143762e+07,3109.556560,3101.243924,4262.403672,142.399908,0.000000,"(10110, 68)","{'n': 10000, 'd': 1000, 'a': -100, 'b': 100, '..."
127,JL transform,0,"{'ep': 0.9, 'de': 0.1}",0.354256,0.225016,0.12,8.196896e+06,819.689590,819.652310,845.682855,...,0.330090,"(10000, 10000)",9.921674e+07,9813.723516,9813.595305,12895.616281,436.943763,0.000000,"(10110, 68)","{'n': 10000, 'd': 10000, 'a': -100, 'b': 100, ..."
153,JL transform,0,"{'ep': 0.9, 'de': 0.1}",0.951245,0.197809,0.15,8.186706e+06,818.670627,818.605567,839.318674,...,0.332035,"(10000, 20000)",1.401844e+08,13865.912270,13852.210428,18263.539188,712.729287,0.000000,"(10110, 68)","{'n': 10000, 'd': 20000, 'a': -100, 'b': 100, ..."
179,JL transform,0,"{'ep': 0.9, 'de': 0.1}",0.103228,0.302497,0.15,1.634135e+07,817.067737,816.867522,910.982119,...,0.333200,"(20000, 1000)",6.253519e+07,3109.656471,3102.973679,4246.267218,159.134924,0.000000,"(20110, 68)","{'n': 20000, 'd': 1000, 'a': -100, 'b': 100, '..."
205,JL transform,0,"{'ep': 0.9, 'de': 0.1}",0.848565,0.244398,0.08,1.638576e+07,819.287830,819.275036,852.882299,...,0.330800,"(20000, 10000)",1.978140e+08,9836.596382,9824.279787,13234.271546,484.110552,0.000000,"(20110, 68)","{'n': 20000, 'd': 10000, 'a': -100, 'b': 100, ..."
231,JL transform,0,"{'ep': 0.9, 'de': 0.1}",1.790119,0.198748,0.15,1.639679e+07,819.839417,819.818476,839.702874,...,0.330060,"(20000, 20000)",2.794013e+08,13893.649878,13881.279501,18433.183794,749.796120,0.000000,"(20110, 68)","{'n': 20000, 'd': 20000, 'a': -100, 'b': 100, ..."
25,Nothing,0,{},0.000027,0.420689,0.99,8.210849e+05,821.084850,820.420886,918.643900,...,0.326400,"(1000, 1000)",9.037071e+05,814.150520,819.764984,918.643900,45.279205,0.326400,"(1110, 1000)","{'n': 1000, 'd': 1000, 'a': -100, 'b': 100, 'c..."


In [226]:
grouped_df = first_df.groupby(by=["original_shape"])
grouped_df.apply(lambda a: a[:])[["name", "params", "reduction_time", "accuracy"]]

name  \
original_shape                                      
(1000, 1000)   0    extremely sparse JL transform   
               1    extremely sparse JL transform   
               2    extremely sparse JL transform   
               3    extremely sparse JL transform   
               4    extremely sparse JL transform   
...                                           ...   
(20000, 20000) 281                   JL transform   
               282                   JL transform   
               283                   JL transform   
               284                            PCA   
               285                        Nothing   

                                                         params  \
original_shape                                                    
(1000, 1000)   0                       {'ep': 0.05, 'de': 0.05}   
               1                        {'ep': 0.05, 'de': 0.1}   
               2                        {'ep': 0.1, 'de': 0.05}   
               3                         {'ep': 0.1, 'de': 0.1}   
               4                        {'ep': 0.5, 'de': 0.05}   
...                                                         ...   
(20000, 20000) 281                       {'ep': 0.5, 'de': 0.1}   
               282                      {'ep': 0.9, 'de': 0.05}   
               283                       {'ep': 0.9, 'de': 0.1}   
               284  {'n_components': 201, 'svd_solver': 'auto'}   
               285                                           {}   

                    reduction_time  accuracy  
original_shape                                
(1000, 1000)   0          0.016116      0.97  
               1          0.036851      0.43  
               2          0.022616      0.58  
               3          0.012347      0.63  
               4          0.008589      0.16  
...                            ...       ...  
(20000, 20000) 281        4.314641      0.15  
               282        3.462470      0.13  
               283        2.188996      0.08  
               284       72.678207      0.95  
               285        0.000028      1.00  

[286 rows x 4 columns]

In [227]:
red_w = 200
ac_w = 5
train_w = 10

In [228]:
score_series = first_df[first_df.name != "Nothing"][["reduction_time", "train_time", "accuracy"]].apply(lambda x: score(x["reduction_time"], x["accuracy"], x["train_time"], red_w, ac_w, train_w), axis=1)
first_df_score = first_df.copy()
first_df_score["score_series"] = score_series
max_ind = first_df_score.groupby(by=["original_shape"])["score_series"].idxmax()
first_df_score.iloc[max_ind][["name", "original_shape", "params", "reduction_time", "accuracy", "train_time", "score_series"]]

,name,original_shape,params,reduction_time,accuracy,train_time,score_series
0,extremely sparse JL transform,"(1000, 1000)","{'ep': 0.05, 'de': 0.05}",0.016116,0.97,0.638783,180.334145
26,extremely sparse JL transform,"(1000, 10000)","{'ep': 0.05, 'de': 0.05}",0.131164,0.99,0.765474,199.707910
54,extremely sparse JL transform,"(1000, 20000)","{'ep': 0.1, 'de': 0.05}",0.234587,0.89,0.418539,117.265248
78,extremely sparse JL transform,"(10000, 1000)","{'ep': 0.05, 'de': 0.05}",0.375906,0.22,2.095778,0.104791
128,PCA,"(10000, 10000)","{'n_components': 101, 'svd_solver': 'auto'}",10.871572,0.99,0.245962,26.800629
131,extremely sparse JL transform,"(10000, 20000)","{'ep': 0.05, 'de': 0.1}",2.695430,0.93,2.415507,48.698186
157,extremely sparse JL transform,"(20000, 1000)","{'ep': 0.05, 'de': 0.1}",0.759569,0.27,2.940273,0.291761
206,PCA,"(20000, 10000)","{'n_components': 201, 'svd_solver': 'auto'}",32.290367,0.99,0.367887,15.273477
224,JL transform,"(20000, 20000)","{'ep': 0.05, 'de': 0.05}",15.108965,0.97,1.388956,15.027845


In [229]:
sh = "(1000, 1000)"
name = "Nothing"
t = first_df[first_df.name == name][first_df.original_shape == sh][["name", "original_shape", "transformed_shape", "params", "reduction_time", "accuracy", "train_time"]]
t["score"] = first_df[first_df.name == name][first_df.original_shape == sh][["name", "original_shape", "transformed_shape", "params", "reduction_time", "accuracy", "train_time"]].apply(lambda x: score(x["reduction_time"], x["train_time"], x["accuracy"]), axis=1)
t

C:\Users\bhum1\anaconda3\envs\datasci-proj\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\bhum1\anaconda3\envs\datasci-proj\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


,name,original_shape,transformed_shape,params,reduction_time,accuracy,train_time,score
25,Nothing,"(1000, 1000)","(1110, 1000)",{},0.000027,0.99,0.420689,92.551607


In [251]:
red_w = 10
ac_w = 10
train_w = 10
included_cols = ["name", "filename", "original_shape", "transformed_shape", "params", "reduction_time", "accuracy", "train_time", "score_series"]

In [252]:
out_df = pd.DataFrame(columns=included_cols)
for res in df_each:
    res_copy = res[0].copy()
    res_copy["filename"] = res[1]
    score_series = res_copy[res_copy.name != "Nothing"][["reduction_time", "train_time", "accuracy"]].apply(lambda x: score(x["reduction_time"], x["accuracy"], red_w, ac_w, train_w), axis=1)
    res_copy_score = res_copy.copy()
    res_copy_score["score_series"] = score_series
    max_ind = res_copy_score.groupby(by=["original_shape"])["score_series"].idxmax()
    collected = res_copy_score.iloc[max_ind][["name", "filename", "original_shape", "transformed_shape", "params", "reduction_time", "accuracy", "train_time", "score_series"]]
    out_df = pd.concat([out_df, collected])

In [253]:
grouped = out_df.groupby(by=["filename"]).apply(lambda a: a[:])
grouped

name  \
filename                                                   
1000_0.33_(-100, 100) 0    extremely sparse JL transform   
                      26   extremely sparse JL transform   
                      68                    JL transform   
                      78   extremely sparse JL transform   
                      122                   JL transform   
...                                                  ...   
500_0_(-500, 500)     116            sparse JL transform   
                      143            sparse JL transform   
                      156  extremely sparse JL transform   
                      197            sparse JL transform   
                      231                   JL transform   

                                        filename  original_shape  \
filename                                                           
1000_0.33_(-100, 100) 0    1000_0.33_(-100, 100)    (1000, 1000)   
                      26   1000_0.33_(-100, 100)   (1000, 10000)   
                      68   1000_0.33_(-100, 100)   (1000, 20000)   
                      78   1000_0.33_(-100, 100)   (10000, 1000)   
                      122  1000_0.33_(-100, 100)  (10000, 10000)   
...                                          ...             ...   
500_0_(-500, 500)     116      500_0_(-500, 500)  (10000, 10000)   
                      143      500_0_(-500, 500)  (10000, 20000)   
                      156      500_0_(-500, 500)   (20000, 1000)   
                      197      500_0_(-500, 500)  (20000, 10000)   
                      231      500_0_(-500, 500)  (20000, 20000)   

                          transformed_shape                    params  \
filename                                                                
1000_0.33_(-100, 100) 0        (1110, 2351)  {'ep': 0.05, 'de': 0.05}   
                      26       (1110, 3135)  {'ep': 0.05, 'de': 0.05}   
                      68       (1110, 2000)  {'ep': 0.05, 'de': 0.05}   
                      78      (10110, 2961)  {'ep': 0.05, 'de': 0.05}   
                      122     (10110, 7189)   {'ep': 0.1, 'de': 0.05}   
...                                     ...                       ...   
500_0_(-500, 500)     116      (10110, 287)   {'ep': 0.5, 'de': 0.05}   
                      143      (10110, 221)    {'ep': 0.5, 'de': 0.1}   
                      156     (20110, 3151)  {'ep': 0.05, 'de': 0.05}   
                      197       (20110, 68)    {'ep': 0.9, 'de': 0.1}   
                      231       (20110, 68)    {'ep': 0.9, 'de': 0.1}   

                           reduction_time  accuracy  train_time  score_series  
filename                                                                       
1000_0.33_(-100, 100) 0          0.016116      0.97    0.638783      8.715012  
                      26         0.131164      0.99    0.765474     10.688152  
                      68         2.809123      0.99    0.450133      4.658938  
                      78         0.375906      0.22    2.095778      0.000003  
                      122       14.604988      1.00    2.349191      2.484848  
...                                   ...       ...         ...           ...  
500_0_(-500, 500)     116        0.879084      1.00    0.275562     11.818182  
                      143        1.876637      1.00    0.330760      6.818182  
                      156        0.688828      1.00    2.663622     11.818182  
                      197        0.917972      0.98    0.267323      9.656315  
                      231        1.733602      1.00    0.281216      6.818182  

[324 rows x 9 columns]

In [254]:
grouped.to_csv("kmeans/kmeans_res_1.csv")

In [255]:
out_df.groupby(["filename", "name"]).count().sort_values(by="original_shape").groupby(level=0).tail(1).sort_values(by="filename")


,,original_shape,transformed_shape,params,reduction_time,accuracy,train_time,score_series
filename,name,,,,,,,
"1000_0.33_(-100, 100)",extremely sparse JL transform,5,5,5,5,5,5,5
"1000_0.33_(-1000, 1000)",extremely sparse JL transform,5,5,5,5,5,5,5
"1000_0.33_(-500, 500)",extremely sparse JL transform,5,5,5,5,5,5,5
"1000_0.66_(-100, 100)",extremely sparse JL transform,6,6,6,6,6,6,6
"1000_0.66_(-1000, 1000)",extremely sparse JL transform,5,5,5,5,5,5,5
"1000_0.66_(-500, 500)",extremely sparse JL transform,4,4,4,4,4,4,4
"1000_0.99_(-100, 100)",extremely sparse JL transform,4,4,4,4,4,4,4
"1000_0.99_(-1000, 1000)",JL transform,5,5,5,5,5,5,5
"1000_0.99_(-500, 500)",extremely sparse JL transform,3,3,3,3,3,3,3
